<a href="https://colab.research.google.com/github/cantaruttim/adai_college/blob/main/Modelagem_College.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import re
from datetime import datetime, timedelta

In [2]:
def valida_arquivo(arquivo):
    extensao = os.path.splitext(arquivo)[1].lower()

    if extensao == ".csv":
        return pd.read_csv(f"{arquivo}", sep=";")
    elif extensao == ".xlsx":
        return pd.read_excel(f"{arquivo}")
    else:
        raise ValueError("Extensão de arquivo não suportada. Use .csv ou .xlsx")


# Modelagem das Bases

## Gera a Base das Turmas

In [67]:
def gera_base_cursos():
  cursos = {
      "cd_curso": ["1", "2"],
      "nome_curso": ["College Intensivo", "College Master"]
  }

  cursos = pd.DataFrame(cursos)
  return cursos

cursos = gera_base_cursos()
cursos

,cd_curso,nome_curso
0,1,College Intensivo
1,2,College Master


## Gera a Base de alunos

In [26]:
def formatar_whatsapp(numero):
    if pd.isnull(numero):
        return numero
    numeros = re.sub(r"\D", "", numero)

    # Verifica se tem 11 dígitos (formato brasileiro com DDD + 9 dígitos)
    if len(numeros) == 11:
        ddd = numeros[:2]
        parte1 = numeros[2:7]
        parte2 = numeros[7:]
        return f"({ddd}) {parte1}-{parte2}"
    else:
        # Retorna como está se não tiver o padrão esperado
        return numero

def gerar_matricula(turma, telefone):
    turma_str = str(turma)
    primeiros_4 = turma_str[:4]
    ultimos_2 = turma_str[-2:]

    telefone_numeros = re.sub(r'\D', '', str(telefone))
    ultimos_4_telefone = telefone_numeros[-4:] if len(telefone_numeros) >= 4 else telefone_numeros

    resultado = primeiros_4 + ultimos_2 + ultimos_4_telefone

    return resultado


def gera_base_alunos(df):
    arquivo = "alunos_pb"

    df["Nome"] = df["Nome"].str.strip()
    df["Email"] = df["Email"].str.strip()

    df["Whatssap"] = df["Whatssap"].apply(formatar_whatsapp)
    df["Whatssap"] = df["Whatssap"].str.strip()

    # Aplicando a função a uma nova coluna
    df['Matricula'] = df.apply(
        lambda row:
                gerar_matricula(
                  row['Turma'],
                  row['Whatssap']
                ),
            axis=1
    )

    df["Matricula"] = df["Matricula"].astype(str)
    df["Matricula"] = pd.to_numeric(df["Matricula"], errors='coerce').fillna(0).astype(int)

    qtd_alunos_turma = (
        df.groupby("Turma")
          .size()
          .reset_index(name="qtdAlunosTurma")
    )

    df = df.merge(qtd_alunos_turma, on="Turma", how="left")

    df["qtdAlunosTurma"] = df["qtdAlunosTurma"].astype(int)

    df.to_excel(f"{arquivo}.xlsx", index=False)

df = pd.read_csv("/content/alunos.csv", sep=";")
gera_base_alunos(df)

In [42]:
alunos = pd.read_excel("/content/alunos_pb.xlsx")
alunos

,Nome,Email,Whatssap,Turma,Matricula,qtdAlunosTurma
0,Abner Cândido Ferreira,abner_ferrer@hotmail.com,(11) 94077-5262,2024.02,2024025262,29
1,Amanda Alfieri Poffo,amandapoffo@gmail.com,(11) 98389-9553,2024.02,2024029553,29
2,Avelino Martins dos Santos Neto,lino__ms@hotmail.com,(11) 94009-7865,2024.02,2024027865,29
3,Beatriz Silva Godoy Marques,godoy.beatriiz@hotmail.com,(11) 98732-3226,2024.02,2024023226,29
4,Cassia Marques Nicastro Medeiros,cassia.nicastro@gmail.com,(11) 97274-4070,2024.02,2024024070,29
...,...,...,...,...,...,...
84,Thiago da Silva Venturi Carvalho,thiagoc.sistemas@gmail.com,(11) 97400-4767,2025.01,2025014767,60
85,Thiago Orsomarso da Silva,thiago.orsomarso@gmail.com,(11) 98735-4944,2025.01,2025014944,60
86,Vanuza Barros Rodrigues de Oliveira,vanuzabroliveira@gmail.com,(11) 96715-4130,2025.01,2025014130,60
87,Vitor Palmarin,vitor_palmarin@hotmail.com,(13) 98822-3252,2025.01,2025013252,60


## Gera a base Módulo

In [66]:
meses_pt_en = {
    "jan": "Jan", "fev": "Feb", "mar": "Mar", "abr": "Apr", "mai": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug", "set": "Sep", "out": "Oct", "nov": "Nov", "dez": "Dec"
}

def data_valida(data_str, formato="%Y-%m-%d"):
    try:
        data = datetime.strptime(data_str, formato)
    except ValueError:
        return False  # Data inválida ou formato errado

    hoje = datetime.today()
    limite = data + timedelta(days=7)
    return hoje <= limite

def traduzir_mes(data_str):
    if not data_str:
        return None
    for pt, en in meses_pt_en.items():
        data_str = data_str.lower().replace(f"/{pt}", f"/{en}")
    return data_str


def retorna_modulo(modulo):
  modulo["status_data"] = modulo["DATA"].apply(lambda x: data_valida(x))

  modulo["DATA"] = modulo["DATA"].fillna("").str.strip()
  modulo["DATA"] = modulo["DATA"].apply(
      lambda x: f"{x}/2025" if x else None
  )

  modulo["DATA"] = modulo["DATA"].apply(traduzir_mes)
  modulo["DATA"] = pd.to_datetime(modulo["DATA"], format="%d/%b/%Y", errors="coerce")

  # Extrai o ano
  modulo["Ano"] = modulo["DATA"].dt.year

  modulo = modulo[["DATA", "Ano", "DIA DA SEMANA", "ATIVIDADE", "MÓDULO", "Semestre", "status_data"]]


  modulo2 = (
      modulo.groupby("Semestre")
        .size()
        .reset_index(name="aulas_por_semestre")
  )

  # Cria a coluna 'ANOMES' no formato YYYYMM
  modulo['ANOMES'] = modulo['DATA'].dt.strftime('%Y%m')

  # Agrupa por 'ANOMES' e 'módulo' e conta a quantidade de aulas
  # Corrigindo o nome da coluna de 'módulo' para 'MÓDULO'
  aulas_por_anomes_modulo = (
      modulo
        .groupby(['ANOMES', 'MÓDULO'])
        .size()
        .reset_index(name='quantidade_aulas')
  )

  modulo = modulo.merge(modulo2,
                        on="Semestre",
                        how="left"
  )

  modulo = modulo.merge(
      aulas_por_anomes_modulo,
      on=['ANOMES', 'MÓDULO'],
      how="left"
  )

  # Corrigindo a lista de nomes de colunas para incluir 'ANOMES'
  modulo.columns = ["data",
                    "ano",
                    "dia_semana",
                    "atividade",
                    "módulo",
                    "semestre",
                    "status_data",
                    "ANOMES",
                    "aulas_por_semestre",
                    "quantidade_aulas"]

  return modulo

modulo = retorna_modulo(pd.read_csv("/content/Módulos.csv", sep=";"))
modulo

,data,ano,dia_semana,atividade,módulo,semestre,status_data,ANOMES,aulas_por_semestre,quantidade_aulas
0,2025-06-03,2025,Terça,"Aula 1- Recapitulando: proto-história, pré-his...",Livros Históricos,Primeiro,False,202506,16,16
1,2025-06-03,2025,Terça,Aula 2 - Recapitulando: os cinco livros da Torah,Livros Históricos,Primeiro,False,202506,16,16
2,2025-06-05,2025,Quinta,Aula 3 - O livro de Josué: A conquista da Terr...,Livros Históricos,Primeiro,False,202506,16,16
3,2025-06-05,2025,Quinta,Aula 4 - O livro de Josué: Repartindo a Herança,Livros Históricos,Primeiro,False,202506,16,16
4,2025-06-10,2025,Terça,Aula 5 - Os Juízes: análise dos livramentos da...,Livros Históricos,Primeiro,False,202506,16,16
5,2025-06-10,2025,Terça,Aula 6 - Os Juízes: um ciclo vicioso de idolat...,Livros Históricos,Primeiro,False,202506,16,16
6,2025-06-12,2025,Quinta,Aula 7 - Transição para monarquia: análise da ...,Livros Históricos,Primeiro,False,202506,16,16
7,2025-06-12,2025,Quinta,Aula 8 - Transição para monarquia: o desenvolv...,Livros Históricos,Primeiro,False,202506,16,16
8,2025-06-17,2025,Terça,Aula 9 - Monarquia Unida: O reinado de Saul e ...,Livros Históricos,Primeiro,False,202506,16,16
9,2025-06-17,2025,Terça,Aula 11 - Reino Separado: análise do contexto ...,Livros Históricos,Primeiro,False,202506,16,16
